In [1]:
# 240630

from glob import glob
import pickle
import os
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor
from itertools import product
from anthropic import AnthropicVertex
import json
from torch.utils.data import Dataset, DataLoader
from format import *

REGION = 'us-east5'
PROJECT_ID = "phonic-impact-421908"
ENDPOINT = f"https://{REGION}-aiplatform.googleapis.com"

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
client = AnthropicVertex(region=REGION, project_id=PROJECT_ID)

In [2]:
class ModelResponder:
    def __init__(self, model_path, ques_path_list, context_list, prompt_func_list=None, path=None):
        self.batch_size = 100
        self.model_path = model_path
        if path is not None:
            self.path = path
        else:
            self.path = os.path.basename(model_path)

        if not prompt_func_list:
            default_prompt_func = lambda inst,ques: f"{ques}"
            prompt_func_list = [default_prompt_func]
        self.prompt_func_list = prompt_func_list

        indexed_ques_paths = list(enumerate(ques_path_list))
        indexed_prompt_funcs = list(enumerate(self.prompt_func_list))
        indexed_contexts = list(enumerate(context_list))
        self.combinations = list(product(indexed_ques_paths, indexed_prompt_funcs, indexed_contexts))

    def process_and_update(self, ques_dict, context, prompt_func, pbar):
        max_retries = 50
        retries = 0
        ques=ques_dict['input']
        while retries < max_retries:
            try:
                responses = client.messages.create(
                    messages=[{"role":"user","content": ques}],
                    system=context,
                    model=self.model_path,
                    temperature=0,
                    max_tokens=1024
                )
                output = responses.to_dict()['content'][0]['text']
                ques_dict['response'] = output
                pbar.update(1)
                return ques_dict
            except Exception as e:
                current = f"{ques_dict['year']}-{ques_dict['session']}-{ques_dict['question_number']}"
                if "Quota" or "quota" in str(e):
                    time.sleep(60)
                else:
                    retries += 1
                    print(e)
                    print(f"Retrying '{current}'... {retries}/{max_retries}")
                    time.sleep(10)
                if retries == max_retries:
                    print(f"Failed to generate response for '{current}', skipping...")
                    ques_dict['response'] = None
                    return ques_dict

    def process_files(self):
        for (ques_idx, ques_path), (prompt_idx, prompt_func), (context_idx, inst) in self.combinations:
            with open(ques_path) as f:
                exam = json.load(f)
            filename=f'output/{self.path} [f{prompt_idx+1}_p{context_idx+1}_q{ques_idx+1}].json'
            dataset = ExamDataset(exam, inst, prompt_func)
            dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)
            with tqdm(total=len(dataloader),desc=filename, leave=True, position=1) as pbar:
                for i, batch in enumerate(dataloader):
                    with ThreadPoolExecutor() as executor:
                        with tqdm(total=len(batch), desc=f"Batch {i}", leave=True, position=0) as batch_pbar:
                            results = list(executor.map(lambda ques: self.process_and_update(ques, inst, prompt_func, batch_pbar), batch))
                    if os.path.exists(filename):
                        with open(filename, 'r') as file:
                            resp = json.load(file)
                    else:
                        resp = []
                        os.makedirs(os.path.dirname(filename), exist_ok=True)
                    for result in results:
                        resp.append(result)
                    with open(filename, 'w', encoding='utf-8') as f:
                        json.dump(resp, f, indent=4, ensure_ascii=False)
                    pbar.update(1)

In [3]:
model_path = "claude-3-5-sonnet@20240620"
run = ModelResponder(model_path, exam_list, inst_list)
run.process_files()

Batch 20: 100%|██████████| 100/100 [00:09<00:00, 10.97it/s]██████▌| 20/21 [03:16<00:09,  9.74s/it]
output/claude-3-5-sonnet@20240620 [f1_p1_q1].json: 100%|██████████| 21/21 [03:25<00:00,  9.80s/it]
Batch 20: 100%|██████████| 100/100 [00:09<00:00, 10.53it/s]██████▌| 20/21 [03:31<00:10, 10.68s/it]
output/claude-3-5-sonnet@20240620 [f1_p1_q2].json: 100%|██████████| 21/21 [03:40<00:00, 10.52s/it]


In [4]:
model_path = "claude-3-opus@20240229"
run = ModelResponder(model_path, exam_list, inst_list)
run.process_files()

Batch 0:   0%|          | 0/100 [00:00<?, ?it/s] 0%|          | 0/21 [00:00<?, ?it/s]

Batch 0:  52%|█████▏    | 52/100 [01:17<00:11,  4.22it/s]     | 0/21 [00:00<?, ?it/s]

Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}
Retrying '2019-1-47'... 1/50
Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}
Retrying '2019-1-28'... 1/50
Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}
Retrying '2019-1-32'... 1/50


Batch 0:  57%|█████▋    | 57/100 [01:19<00:19,  2.26it/s]

Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}
Retrying '2019-1-33'... 1/50


...

Batch 4:  61%|██████    | 61/100 [01:19<00:10,  3.68it/s]     | 4/21 [08:17<33:36, 118.60s/it]

Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}
Retrying '2020-2-3'... 1/50


Batch 4:  75%|███████▌  | 75/100 [01:24<00:10,  2.35it/s]

Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}
Retrying '2020-2-22'... 1/50
Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}
Retrying '2020-2-28'... 1/50


output/claude-3-opus@20240229 [f1_p1_q2].json: 100%|██████████| 21/21 [41:47<00:00, 119.41s/it]


In [3]:
model_path = "claude-3-sonnet@20240229"
run = ModelResponder(model_path, exam_list, inst_list)
run.process_files()

Batch 0:   0%|          | 0/100 [00:00<?, ?it/s]